In [1]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf
import PIL.ImageOps as ImageOps
import PIL.Image as Image
from tqdm import tqdm

In [2]:
class_names = ['Arive-Dantu',
                'Basale',
               'Betel',
               'Crape_Jasmine',
               'Curry', 'Drumstick',
               'Fenugreek',
               'Guava',
               'Hibiscus',
               'Indian_Beech',
               'Indian_Mustard',
               'Jackfruit',
               'Jamaica_Cherry-Gasagase',
               'Jamun',
               'Jasmine',
               'Karanda',
               'Lemon',
               'Mango',
               'Mexican_Mint',
               'Mint',
               'Neem',
               'Oleander',
               'Parijata',
               'Peepal',
               'Pomegranate',
               'Rasna',
               'Rose_apple',
               'Roxburgh_fig',
               'Sandalwood',
               'Tulsi']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (64,64)

In [3]:
def pre_process(img_path):
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, IMAGE_SIZE) 
    return image

In [4]:
def load_data():
    
    datasets = ['C:/Users/sanju/OneDrive/Documents/Nitheesh/new/segment','C:/Users/sanju/OneDrive/Documents/Nitheesh/Test']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = pre_process(img_path) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [5]:
(train_images, train_labels), (test_images, test_labels) = load_data()

Loading C:/Users/sanju/OneDrive/Documents/Nitheesh/new/segment


100%|██████████| 500/500 [00:01<00:00, 470.19it/s]


Loading C:/Users/sanju/OneDrive/Documents/Nitheesh/Test


100%|██████████| 99/99 [00:00<00:00, 420.80it/s]


In [6]:
train_images = train_images / 255.0 
test_images = test_images / 255.0

In [18]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
model = tf.keras.Sequential([
    # Layer 1
    tf.keras.layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(),
    
    # Layer 2
    tf.keras.layers.Conv2D(256, (5, 5), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(),
    
    # Layer 3
    tf.keras.layers.Conv2D(384, (3, 3), activation='relu', padding='same'),
    
    # Layer 4
    tf.keras.layers.Conv2D(384, (3, 3), activation='relu', padding='same'),
    
    # Layer 5
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(),
    
    # Flatten
    tf.keras.layers.Flatten(),
    
    # Layer 6
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Layer 7
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Layer 8 (Output layer)
    tf.keras.layers.Dense(30, activation='softmax')
])


ValueError: Exception encountered when calling layer "max_pooling2d_20" (type MaxPooling2D).

Negative dimension size caused by subtracting 3 from 2 for '{{node max_pooling2d_20/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 3, 3, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,2,2,256].

Call arguments received by layer "max_pooling2d_20" (type MaxPooling2D):
  • inputs=tf.Tensor(shape=(None, 2, 2, 256), dtype=float32)

In [9]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_split = 0.2)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\sanju\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\sanju\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sanju\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\sanju\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\sanju\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   